In [1]:

import pandas as pd
import os

# --- Fonction utilitaire ---
def test_import(nom_fichier, sep=',', nrows=5):
    """
    Teste le chargement d'un fichier CSV :
    - Vérifie s'il existe
    - Essaie de lire les premières lignes
    - Affiche la taille, les colonnes, et un extrait
    """
    print("----------------------------------------------------")
    print(f"📄 Test du fichier : {nom_fichier}")

    if not os.path.exists(nom_fichier):
        print("❌ Fichier introuvable.\n")
        return

    try:
        df = pd.read_csv(nom_fichier, sep=sep, nrows=nrows, low_memory=False)
        print(f"✅ Chargement réussi ({df.shape[0]} lignes × {df.shape[1]} colonnes)")
        print("   ➜ Colonnes :", list(df.columns[:10]))
        print(df.head(2), "\n")
    except Exception as e:
        print(f"⚠️ Erreur lors du chargement : {e}\n")


dvf_files = [f"data/dvf_{dep}.csv" for dep in ["75", "77", "78", "91", "92", "93", "94", "95"]]

print("🔍 Vérification des fichiers DVF départementaux...\n")
for fichier in dvf_files:
    test_import(fichier, sep=",")

autres_bases = [
    {"nom": "INSEE - Données socio-économiques", "path": "data/insee_dossier_complet.csv", "sep": ";"},
    {"nom": "DPE - Diagnostic énergétique", "path": "data/dpe_logement.csv", "sep": ","},
    {"nom": "Airparif - Qualité de l’air", "path": "data/air_parif_communes.csv", "sep": ";"},
    {"nom": "Bruitparif - Niveau sonore", "path": "data/bruitsparifs_communes.csv", "sep": ";"},
    {"nom": "Encadrement des loyers (Paris)", "path": "data/encadrement_des_loyers_paris.csv", "sep": ","},
    {"nom": "Délinquance - Sécurité", "path": "data/delinquance_communes.csv", "sep": ","},
    {"nom": "Transports - Île-de-France", "path": "data/transports_idf.csv", "sep": ";"},
    {"nom": "Éducation - Annuaire établissements", "path": "data/fr-en-annuaire-education.csv", "sep": ";"},
    {"nom": "Éducation - Principaux établissements", "path": "data/fr-en-principaux-etablissement.csv", "sep": ";"}
]

print("\n🔍 Vérification des bases complémentaires...\n")
for base in autres_bases:
    print(f"🔹 {base['nom']}")
    test_import(base["path"], sep=base["sep"])


🔍 Vérification des fichiers DVF départementaux...

----------------------------------------------------
📄 Test du fichier : data/dvf_75.csv
❌ Fichier introuvable.

----------------------------------------------------
📄 Test du fichier : data/dvf_77.csv
❌ Fichier introuvable.

----------------------------------------------------
📄 Test du fichier : data/dvf_78.csv
❌ Fichier introuvable.

----------------------------------------------------
📄 Test du fichier : data/dvf_91.csv
❌ Fichier introuvable.

----------------------------------------------------
📄 Test du fichier : data/dvf_92.csv
❌ Fichier introuvable.

----------------------------------------------------
📄 Test du fichier : data/dvf_93.csv
❌ Fichier introuvable.

----------------------------------------------------
📄 Test du fichier : data/dvf_94.csv
❌ Fichier introuvable.

----------------------------------------------------
📄 Test du fichier : data/dvf_95.csv
❌ Fichier introuvable.


🔍 Vérification des bases complémentaires...


In [ ]:
from pathlib import Path
import pandas as pd

# 1) Où écrire le fichier fusionné ?
# Choisis l'un des deux chemins de sortie (dé-commente celui que tu veux garder) :
# out_path = Path("data/raw/dvf_idf.csv")   # si tu as ce dossier
out_path = Path("dvf_idf.csv")              # à la racine du projet (même dossier que tes dvf_75.csv, etc.)

out_path.parent.mkdir(parents=True, exist_ok=True)

# 2) Trouver automatiquement les fichiers DVF départementaux
here = Path(".")
dvf_files = list(here.glob("dvf_*.csv"))
if not dvf_files:
    # si rien au niveau racine, on cherche en profondeur (sous-dossiers)
    dvf_files = list(here.rglob("dvf_*.csv"))

print("📂 Fichiers DVF trouvés :", len(dvf_files))
for f in dvf_files:
    print("   -", f.relative_to(here))

if not dvf_files:
    raise FileNotFoundError("Aucun fichier 'dvf_*.csv' trouvé. Vérifie les noms/chemins.")

# 3) Lecture robuste (encodage + séparateur)
def robust_read(path):
    try:
        return pd.read_csv(path, sep=";", encoding="utf-8", low_memory=False)
    except Exception:
        try:
            return pd.read_csv(path, sep=";", encoding="latin-1", low_memory=False)
        except Exception:
            # dernier essai: on tente séparateur virgule
            try:
                return pd.read_csv(path, sep=",", encoding="utf-8", low_memory=False)
            except Exception:
                return pd.read_csv(path, sep=",", encoding="latin-1", low_memory=False)

dfs = []
for f in dvf_files:
    df = robust_read(f)
    # Normalisation minimale utile pour DVF
    df.columns = [c.strip().lower().replace(" ", "_") for c in df.columns]
    dfs.append(df)

# 4) Fusion
dvf_all = pd.concat(dfs, ignore_index=True)
print("✅ Fusion réalisée :", dvf_all.shape)

# 5) Sauvegarde
dvf_all.to_csv(out_path, index=False)
print("💾 Fichier fusionné sauvegardé sous :", out_path.resolve())


📂 Fichiers DVF trouvés : 7
   - data/clean/dvf_75_clean.csv
   - data/clean/dvf_77_clean.csv
   - data/clean/dvf_91_clean.csv
   - data/clean/dvf_92_clean.csv
   - data/clean/dvf_93_clean.csv
   - data/clean/dvf_94_clean.csv
   - data/clean/dvf_95_clean.csv


In [ ]:
import pandas as pd
print("🔍 Test air_parif_communes.csv")

try:
    df_air = pd.read_csv(
        "data/air_parif_communes.csv",
        sep=',',
        encoding='utf-8',
        engine='python'
    )
    print("✅ Chargement réussi :", df_air.shape)
    display(df_air.head(5))
except Exception as e:
    print("⚠️ Erreur UTF-8 :", e)
    print("🔁 Nouvelle tentative avec encodage latin-1 et séparateur ';'")
    try:
        df_air = pd.read_csv(
            "data/air_parif_communes.csv",
            sep=';',
            encoding='latin-1',
            engine='python'
        )
        print("✅ Chargement réussi (latin-1) :", df_air.shape)
        display(df_air.head(5))
    except Exception as e2:
        print("❌ Toujours erreur :", e2)


In [ ]:
# === ETAPE 1 : SETUP + VERIFICATIONS AUTOMATIQUES ===
# Objectif : vérifier l'environnement, préparer les dossiers, détecter les CSV, et tester une lecture légère

import sys, subprocess
from pathlib import Path

def ensure(pkg):
    try:
        __import__(pkg)
        print(f"✅ {pkg} déjà installé")
    except ImportError:
        print(f"⏳ Installation de {pkg}…")
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])
        print(f"✅ {pkg} installé")

# 1) Dépendances minimales
for p in ["pandas", "numpy"]:
    ensure(p)

import pandas as pd
import numpy as np

pd.set_option("display.max_columns", 120)
pd.set_option("display.width", 160)

print("\n📦 Versions")
print(" - Python :", sys.version.split()[0])
print(" - pandas :", pd.__version__)
print(" - numpy  :", np.__version__)

# 2) Dossiers projet
BASE_DIR = Path(".").resolve()
DATA_DIR = BASE_DIR / "data"
CLEAN_DIR = DATA_DIR / "clean"
DATA_DIR.mkdir(parents=True, exist_ok=True)
CLEAN_DIR.mkdir(parents=True, exist_ok=True)

print("\n📁 Dossiers")
print(" - Projet :", BASE_DIR)
print(" - Données :", DATA_DIR)
print(" - Nettoyés:", CLEAN_DIR)

# 3) Détection automatique de TOUS les CSV (sauf ceux déjà nettoyés)
FILES = {
    f.stem.replace("-", "_"): f
    for f in DATA_DIR.glob("*.csv")
    if not str(f).startswith(str(CLEAN_DIR))
}

print("\n🔎 CSV détectés dans 'data/' (hors 'data/clean/'):")
if not FILES:
    print("   ⚠️ Aucun fichier .csv trouvé dans data/. Place tes fichiers ici, puis relance cette cellule.")
else:
    for k, p in sorted(FILES.items()):
        print(f"   • {k:35s} → {p.name}")

# 4) Mini test de lecture (robuste) sur chaque CSV : 2 lignes max
def test_preview(path: Path):
    encodings = ["utf-8", "utf-8-sig", "latin-1", "cp1252"]
    seps = [";", ",", "\t", "|"]
    last_err = None
    for enc in encodings:
        for sep in seps:
            try:
                df = pd.read_csv(path, sep=sep, encoding=enc, nrows=2, )
                # Heuristique anti faux-positif : si 1 colonne unique très longue -> mauvais séparateur
                if df.shape[1] == 1:
                    continue
                return True, enc, sep, df.columns.tolist()[:6]
            except Exception as e:
                last_err = e
                continue
    return False, None, None, str(last_err)

print("\n🧪 Test de lecture rapide (2 lignes) :")
if FILES:
    ok_all = True
    for k, p in sorted(FILES.items()):
        ok, enc, sep, info = test_preview(p)
        if ok:
            print(f"   ✅ {p.name:35s} | enc='{enc}', sep='{sep}' | colonnes: {info}")
        else:
            ok_all = False
            print(f"   ❌ {p.name:35s} | lecture impossible (dernier message: {info})")
    if ok_all:
        print("\n🎉 ETAPE 1 OK : environnement prêt, dossiers en place, CSV détectés et lisibles.")
    else:
        print("\n⚠️ ETAPE 1 PARTIELLE : certains fichiers ne se lisent pas en preview. On pourra les traiter au cas par cas au nettoyage.")
else:
    print("   (aucun fichier à tester)")

# 5) Expose les variables pour les étapes suivantes
globals()["DATA_DIR"] = DATA_DIR
globals()["CLEAN_DIR"] = CLEAN_DIR
globals()["FILES"] = FILES


In [ ]:
# === ETAPE 2 : DÉCLARATION AUTOMATIQUE DES FICHIERS + SCHÉMAS DE NETTOYAGE ===
# Objectif : identifier tous les fichiers de données dans "data/", définir des schémas adaptés,
# et préparer le pipeline pour le nettoyage (étape 4).

from pathlib import Path

# --- Réutilisation des dossiers de l'étape 1
DATA_RAW = DATA_DIR
DATA_CLEAN = CLEAN_DIR

# === 1️⃣ Détection automatique des fichiers CSV ===
FILES = {
    f.stem.replace("-", "_"): f
    for f in DATA_RAW.glob("*.csv")
    if not str(f).startswith(str(DATA_CLEAN))  # on ignore les fichiers déjà nettoyés
}

print("📂 Fichiers détectés automatiquement :")
for key, path in sorted(FILES.items()):
    print(f"   • {key:30s} → {path.name}")
print(f"\nTotal : {len(FILES)} fichiers détectés dans {DATA_RAW}\n")


# === 2️⃣ Schémas personnalisés pour certaines bases connues ===
SCHEMAS_CUSTOM = {
    "dvf_idf": {
        "rename": {
            "valeur_fonciere": "valeur_fonciere",
            "surface_reelle_bati": "surface_reelle_bati",
            "type_local": "type_local",
            "nombre_pieces_principales": "nb_pieces",
            "date_mutation": "date_mutation",
            "commune": "commune",
        },
        "dtype": {
            "valeur_fonciere": "float",
            "surface_reelle_bati": "float",
            "nb_pieces": "Int64",
        },
        "parse_dates": ["date_mutation"],
        "drop_dupes_on": ["code_commune", "date_mutation", "valeur_fonciere", "surface_reelle_bati"],
    },
    "delinquance_communes": {
        "rename": {
            "faits_total": "faits_total",
            "population": "population",
            "annee": "annee",
            "commune": "commune",
            "taux_criminalite": "taux_criminalite",
        },
        "dtype": {
            "faits_total": "Int64",
            "population": "Int64",
            "annee": "Int64",
        },
        "drop_dupes_on": ["code_commune", "annee"],
    },
    "air_parif_communes": {
        "rename": {"indice_airparif": "indice_airparif", "commune": "commune"},
        "dtype": {"indice_airparif": "float"},
        "drop_dupes_on": ["code_commune"],
    },
    "dpe_logement": {
        "rename": {"date_visite_diagnostiqueur": "date_dpe", "commune": "commune"},
        "dtype": {},
        "drop_dupes_on": ["code_commune"],
    },
    "insee_dossier_complet": {
        "rename": {
            "revenu_median": "revenu_median",
            "taux_chomage": "taux_chomage",
            "population": "population",
            "commune": "commune",
        },
        "dtype": {"revenu_median": "float", "taux_chomage": "float", "population": "Int64"},
        "drop_dupes_on": ["code_commune"],
    },
    "transports_idf": {
        "rename": {"nb_arrets": "nb_arrets", "score_connectivite": "score_connectivite", "commune": "commune"},
        "dtype": {"nb_arrets": "Int64", "score_connectivite": "float"},
        "drop_dupes_on": ["code_commune"],
    },
}

# === 3️⃣ Schéma par défaut pour toutes les autres bases ===
DEFAULT_SCHEMA = {
    "rename": {},
    "dtype": {},
    "drop_dupes_on": ["code_commune"],
}

# === 4️⃣ Génération automatique des schémas ===
SCHEMAS = {}
for key in FILES.keys():
    SCHEMAS[key] = SCHEMAS_CUSTOM.get(key, DEFAULT_SCHEMA)

print("📘 Schémas générés :")
for k in sorted(SCHEMAS.keys()):
    base_type = "🎯 personnalisé" if k in SCHEMAS_CUSTOM else "⚙️  générique"
    print(f"   {k:30s} → {base_type}")
print(f"\nTotal : {len(SCHEMAS)} schémas chargés\n")


# === 5️⃣ Vérification rapide de cohérence ===
missing = [k for k in FILES if k not in SCHEMAS]
if missing:
    print("⚠️ Bases sans schéma associé :", missing)
else:
    print("✅ Toutes les bases détectées ont un schéma associé (automatique ou personnalisé).")

# --- Expose les variables pour les étapes suivantes
globals()["FILES"] = FILES
globals()["SCHEMAS"] = SCHEMAS


In [ ]:
# === ETAPE 3 : NETTOYAGE AUTOMATIQUE ULTRA OPTIMISÉ (GROS FICHIERS) ===

import pandas as pd
import numpy as np
import unicodedata
import csv, time, sys
from pathlib import Path
from tqdm.notebook import tqdm

CHUNK_SIZE = 30000  # plus petit = plus fluide
pd.options.display.max_columns = 200

print("🚀 Mode optimisation activé : lecture par petits paquets + écriture directe\n")

def normalize_colnames(cols):
    def _norm(s):
        s = str(s).strip().lower()
        s = "".join(c for c in unicodedata.normalize("NFKD", s) if not unicodedata.combining(c))
        s = s.replace(" ", "_").replace("-", "_").replace("/", "_")
        s = "_".join([token for token in s.split("_") if token])
        return s
    return [_norm(c) for c in cols]

def normalize_commune_name(s):
    if pd.isna(s): return np.nan
    s = str(s).strip().upper()
    s = "".join(c for c in unicodedata.normalize("NFKD", s) if not unicodedata.combining(c))
    s = s.replace("-", " ").replace("'", " ")
    return " ".join(s.split())

def add_commune_keys(df):
    df = df.copy()
    code_col = next((c for c in df.columns if "insee" in c or "code_commune" in c), None)
    if code_col:
        df["code_commune"] = df[code_col].astype(str).str.extract(r"(\d+)")[0].str.zfill(5)
    else:
        df["code_commune"] = np.nan
    name_col = next((c for c in df.columns if "commune" in c or "nom" == c), None)
    if name_col:
        df["commune_std"] = df[name_col].apply(normalize_commune_name)
    else:
        df["commune_std"] = np.nan
    return df

def detect_separator(path):
    try:
        with open(path, encoding="utf-8") as f:
            sample = f.read(2048)
        sep = csv.Sniffer().sniff(sample).delimiter
        if sep not in [";", ",", "\t", "|"]:
            sep = ";"
        return sep
    except Exception:
        return ";"

def clean_one_lightweight(key, path, schema):
    print(f"\n🧩 === {key.upper()} ===")
    print(f"📄 Lecture du fichier : {path.name}")

    sep = detect_separator(path)
    out_path = DATA_CLEAN / f"{key}_clean.csv"
    first_chunk = True
    total_lines = 0
    start = time.time()

    try:
        reader = pd.read_csv(
            path, sep=sep, encoding="utf-8", chunksize=CHUNK_SIZE,
            engine="python", on_bad_lines="skip"
        )

        for chunk in tqdm(reader, desc=f"{key}", unit="chunk"):
            total_lines += len(chunk)
            chunk.columns = normalize_colnames(chunk.columns)
            chunk = add_commune_keys(chunk)

            rename_map = {old: new for old, new in schema.get("rename", {}).items() if old in chunk.columns}
            chunk = chunk.rename(columns=rename_map)

            for col, dtype in schema.get("dtype", {}).items():
                if col in chunk.columns:
                    try:
                        if dtype == "Int64":
                            chunk[col] = pd.to_numeric(chunk[col], errors="coerce").astype("Int64")
                        elif dtype == "float":
                            chunk[col] = pd.to_numeric(chunk[col], errors="coerce")
                    except Exception:
                        pass

            chunk.to_csv(out_path, index=False, mode="a", header=first_chunk)
            first_chunk = False

        elapsed = round(time.time() - start, 2)
        print(f"✅ {key} terminé ({total_lines:,} lignes) en {elapsed}s")
        return {"base": key, "lignes": total_lines, "fichier": str(out_path)}

    except Exception as e:
        print(f"❌ Erreur sur {key}: {e}")
        return {"base": key, "lignes": 0, "fichier": str(path), "erreur": str(e)}


# --- Boucle principale ---
results = []
for key, path in FILES.items():
    if not path.exists():
        print(f"⚠️ Fichier manquant : {path}")
        continue
    schema = SCHEMAS.get(key, {"rename": {}, "dtype": {}})
    res = clean_one_lightweight(key, path, schema)
    results.append(res)

# --- Résumé global ---
print("\n📊 === RÉSUMÉ GLOBAL DU NETTOYAGE ===")
summary = pd.DataFrame(results)
display(summary)

print("\n✅ Étape 3 terminée (mode léger). Aucune surcharge mémoire détectée.")


In [ ]:
# === ETAPE 4 : NETTOYAGE COMPLET ET CONTRÔLÉ ===
# Objectif : lire, uniformiser, nettoyer et sauvegarder toutes les bases de données.

import pandas as pd
import numpy as np
import unicodedata
from pathlib import Path
import os
import shutil

# --- Dossiers
DATA_RAW = Path("data")         # 🔧 fichiers sources
DATA_CLEAN = DATA_RAW / "clean" # 🔧 fichiers nettoyés
DATA_CLEAN.mkdir(parents=True, exist_ok=True)

# --- Nettoyage du dossier clean avant traitement
if DATA_CLEAN.exists():
    print("🧽 Nettoyage du dossier 'data/clean'...")
    for f in DATA_CLEAN.glob("*.csv"):
        try:
            f.unlink()
        except Exception as e:
            print(f"⚠️ Impossible de supprimer {f.name} : {e}")

# --- Fichiers à traiter
FILES = {
    "dvf": DATA_RAW / "dvf_idf.csv",
    "delinquance": DATA_RAW / "delinquance_communes.csv",
    "air": DATA_RAW / "air_parif_communes.csv",
    "encadrement": DATA_RAW / "encadrement_loyers_idf.csv",
    "insee": DATA_RAW / "insee_dossier.csv",
    "dpe": DATA_RAW / "dpe_logements.csv",
    "transport": DATA_RAW / "transports_idf.csv",
    "rne": DATA_RAW / "fr-en-annuaire-education.csv",
}

print("📂 Vérification des fichiers présents :")
for key, path in FILES.items():
    print(f"{key:12s} → {path} {'✅' if path.exists() else '❌'}")

# --- Fonctions utilitaires
def normalize_colnames(cols):
    """Uniformise les noms de colonnes (minuscules, sans accents, snake_case)."""
    def _norm(s):
        s = str(s).strip().lower()
        s = "".join(c for c in unicodedata.normalize("NFKD", s) if not unicodedata.combining(c))
        s = s.replace(" ", "_").replace("-", "_").replace("/", "_")
        s = "_".join([token for token in s.split("_") if token])
        return s
    return [_norm(c) for c in cols]

def normalize_commune_name(s):
    """Normalise les noms de communes pour éviter les erreurs de casse ou d'accents."""
    if pd.isna(s):
        return np.nan
    s = str(s).strip().upper()
    s = "".join(c for c in unicodedata.normalize("NFKD", s) if not unicodedata.combining(c))
    s = s.replace("-", " ").replace("'", " ")
    s = " ".join(s.split())
    return s

def add_commune_keys(df):
    """Crée les colonnes code_commune et commune_std pour les jointures futures."""
    df = df.copy()
    code_col = next((c for c in df.columns if "insee" in c or "code_commune" in c), None)
    if code_col:
        df["code_commune"] = df[code_col].astype(str).str.extract(r"(\d+)")[0].str.zfill(5)
    else:
        df["code_commune"] = np.nan
    name_col = next((c for c in df.columns if "commune" in c or c == "nom"), None)
    if name_col:
        df["commune_std"] = df[name_col].apply(normalize_commune_name)
    else:
        df["commune_std"] = np.nan
    return df

def remove_empty_columns(df):
    """Supprime les colonnes entièrement vides."""
    before = df.shape[1]
    df = df.dropna(axis=1, how="all")
    after = df.shape[1]
    if before != after:
        print(f"🧹 {before - after} colonnes entièrement vides supprimées")
    return df

def add_paris_mapping_if_missing(df, key):
    """Ajoute les codes communes manquants pour les fichiers BruitParif/Paris."""
    if key not in ["bruitsparif", "bruitsparifs", "bruitsparifs_communes"]:
        return df
    if "code_commune" in df.columns and df["code_commune"].notna().sum() > 0:
        return df

    paris_mapping = {
        "1er": 75101, "2e": 75102, "3e": 75103, "4e": 75104, "5e": 75105, "6e": 75106,
        "7e": 75107, "8e": 75108, "9e": 75109, "10e": 75110, "11e": 75111, "12e": 75112,
        "13e": 75113, "14e": 75114, "15e": 75115, "16e": 75116, "17e": 75117,
        "18e": 75118, "19e": 75119, "20e": 75120
    }

    for col in df.columns:
        if df[col].astype(str).str.contains("arrondissement|paris", case=False).any():
            df["code_commune"] = (
                df[col]
                .astype(str)
                .str.extract(r"(\d+)")
                .astype(float)
                .astype("Int64")
                .map(paris_mapping)
            )
            df["commune_std"] = "PARIS"
            print("🏙️ Codes communes ajoutés via mapping Paris")
            break
    return df

# --- Schémas de nettoyage
SCHEMAS = {
    "dvf": {"rename": {"valeur_fonciere": "valeur_fonciere", "surface_reelle_bati": "surface_reelle_bati",
                       "type_local": "type_local", "nombre_pieces_principales": "nb_pieces",
                       "date_mutation": "date_mutation", "commune": "commune"},
            "dtype": {"valeur_fonciere": "float", "surface_reelle_bati": "float", "nb_pieces": "Int64"},
            "parse_dates": ["date_mutation"],
            "drop_dupes_on": ["code_commune", "date_mutation", "valeur_fonciere", "surface_reelle_bati"]},
    "delinquance": {"rename": {"faits_total": "faits_total", "population": "population", "annee": "annee",
                               "commune": "commune", "taux_criminalite": "taux_criminalite"},
                    "dtype": {"faits_total": "Int64", "population": "Int64", "annee": "Int64"},
                    "drop_dupes_on": ["code_commune", "annee"]},
    "air": {"rename": {"indice_airparif": "indice_airparif", "commune": "commune"},
            "dtype": {"indice_airparif": "float"},
            "drop_dupes_on": ["code_commune"]},
    "encadrement": {"rename": {"zone_encadree": "zone_encadree", "loyer_ref": "loyer_ref",
                               "loyer_majoré": "loyer_majore", "commune": "commune"},
                    "dtype": {},
                    "drop_dupes_on": ["code_commune"]},
    "insee": {"rename": {"revenu_median": "revenu_median", "taux_chomage": "taux_chomage",
                         "pop_18_29": "pop_18_29", "population": "population",
                         "logements_vacants": "logements_vacants", "logements_totaux": "logements_totaux",
                         "commune": "commune"},
              "dtype": {"revenu_median": "float", "taux_chomage": "float", "pop_18_29": "Int64",
                        "population": "Int64", "logements_vacants": "Int64", "logements_totaux": "Int64"},
              "drop_dupes_on": ["code_commune"]},
    "dpe": {"rename": {"nb_a": "nb_a", "nb_b": "nb_b", "nb_c": "nb_c", "nb_d": "nb_d", "nb_e": "nb_e",
                       "nb_f": "nb_f", "nb_g": "nb_g", "commune": "commune"},
            "dtype": {k: "Int64" for k in ["nb_a", "nb_b", "nb_c", "nb_d", "nb_e", "nb_f", "nb_g"]},
            "drop_dupes_on": ["code_commune"]},
    "transport": {"rename": {"nb_arrets": "nb_arrets", "score_connectivite": "score_connectivite", "commune": "commune"},
                  "dtype": {"nb_arrets": "Int64", "score_connectivite": "float"},
                  "drop_dupes_on": ["code_commune"]},
    "rne": {"rename": {"nb_etabs_sup": "nb_etabs_sup", "superficie_km2": "superficie_km2", "commune": "commune"},
            "dtype": {"nb_etabs_sup": "Int64", "superficie_km2": "float"},
            "drop_dupes_on": ["code_commune"]},
}

# --- Fonction principale de nettoyage
def clean_one(key, path, schema):
    print(f"\n🧩 === {key.upper()} ===")
    print(f"📄 Lecture du fichier : {path}")

    try:
        df = pd.read_csv(path, sep=";", encoding="utf-8", low_memory=False)
    except UnicodeDecodeError:
        df = pd.read_csv(path, sep=";", encoding="latin-1", low_memory=False)
    except Exception:
        df = pd.read_csv(path, sep=",", encoding="utf-8", low_memory=False)

    print(f"   ➤ {df.shape[0]} lignes, {df.shape[1]} colonnes avant nettoyage")

    # Normalisation et renommage
    df.columns = normalize_colnames(df.columns)
    rename_map = {old: new for old, new in schema.get("rename", {}).items() if old in df.columns}
    df = df.rename(columns=rename_map)

    # Ajout des clés communes et suppression des colonnes vides
    df = add_commune_keys(df)
    df = remove_empty_columns(df)
    df = add_paris_mapping_if_missing(df, key)

    # Typage
    for col, dtype in schema.get("dtype", {}).items():
        if col in df.columns:
            try:
                if dtype == "Int64":
                    df[col] = pd.to_numeric(df[col], errors="coerce").astype("Int64")
                elif dtype == "float":
                    df[col] = pd.to_numeric(df[col], errors="coerce")
            except Exception as e:
                print(f"⚠️ Typage impossible pour {col}: {e}")

    # Suppression des doublons
    subset = [c for c in schema.get("drop_dupes_on", []) if c in df.columns]
    if subset:
        before = len(df)
        df = df.drop_duplicates(subset=subset)
        print(f"🧹 {before - len(df)} doublons supprimés sur {subset}")

    # Export sécurisé
    out = DATA_CLEAN / f"{key}_clean.csv"
    try:
        df.to_csv(out, index=False, encoding="utf-8")
        print(f"✅ Exporté vers {out} ({df.shape[0]} lignes, {df.shape[1]} colonnes)")
    except Exception as e:
        print(f"❌ Erreur lors de l’export de {out} : {e}")

    return {"base": key, "lignes": df.shape[0], "colonnes": df.shape[1], "fichier": str(out)}

# --- Boucle principale de traitement
results = []
for key, path in FILES.items():
    if not path.exists():
        print(f"❌ Fichier manquant : {path}")
        continue
    try:
        with open(path, "r", encoding="utf-8") as f:
            f.read(1024)
    except Exception as e:
        print(f"⚠️ Fichier {path.name} inaccessible : {e}")
        continue

    schema = SCHEMAS.get(key, {"rename": {}, "dtype": {}})
    res = clean_one(key, path, schema)
    results.append(res)

# --- Résumé global
print("\n📊 === RÉSUMÉ GLOBAL DU NETTOYAGE ===")
if results:
    summary = pd.DataFrame(results)
    summary["taille_Ko"] = summary["fichier"].apply(lambda x: round(os.path.getsize(x) / 1024, 1))
    display(summary)
    summary.to_csv(DATA_CLEAN / "rapport_validation.csv", index=False)
    print(f"📝 Rapport sauvegardé dans {DATA_CLEAN / 'rapport_validation.csv'}")
else:
    print("⚠️ Aucun fichier n’a été traité.")


In [ ]:
from pathlib import Path
import pandas as pd
import unicodedata, csv

DATA_RAW   = Path("data")          # dvf_75.csv, dvf_77.csv, etc.
DATA_CLEAN = Path("data/clean")
DATA_CLEAN.mkdir(parents=True, exist_ok=True)

def normalize_colnames(cols):
    def _n(s):
        s = str(s).strip().lower()
        s = "".join(c for c in unicodedata.normalize("NFKD", s) if not unicodedata.combining(c))
        return s.replace(" ", "_").replace("-", "_").replace("/", "_")
    return [_n(c) for c in cols]

def add_commune_keys(df):
    df = df.copy()
    if "code_commune" in df.columns:
        df["code_commune"] = (
            df["code_commune"].astype(str).str.extract(r"(\d+)")[0].str.zfill(5)
        )
    if "nom_commune" in df.columns:
        s = df["nom_commune"].astype(str)
        s = s.str.normalize("NFKD").str.encode("ascii","ignore").str.decode("ascii")
        s = s.str.upper().str.replace("-", " ", regex=False).str.replace("'", " ", regex=False)
        df["commune_std"] = s.str.split().str.join(" ")
    return df

def read_dvf_robuste(src):
    try:
        return pd.read_csv(src, sep=",", encoding="utf-8", low_memory=False)
    except Exception:
        return pd.read_csv(
            src, sep=",", encoding="utf-8",
            engine="python", quotechar='"', on_bad_lines="skip"
        )

# DVF bruts à traiter
raw_dvf = sorted([p for p in DATA_RAW.glob("dvf_*.csv") if "_clean" not in p.name])
print(f"📂 DVF bruts détectés ({len(raw_dvf)}):")
for p in raw_dvf: print("  -", p.name)

for src in raw_dvf:
    dest = DATA_CLEAN / (src.stem + "_clean.csv")
    print(f"\n🧩 {src.name} → {dest.name}")

    df = read_dvf_robuste(src)
    df.columns = normalize_colnames(df.columns)
    df = add_commune_keys(df)

    # ✅ Correction : lineterminator (sans underscore)
    df.to_csv(
        dest, index=False, encoding="utf-8",
        sep=",", quoting=csv.QUOTE_MINIMAL, lineterminator="\n"
    )

    chk = pd.read_csv(dest, sep=",", nrows=3)
    print(f"   ✅ OK : {chk.shape[1]} colonnes | Aperçu colonnes : {list(chk.columns)[:6]}")

print("\n🎉 Recréation des DVF _clean terminée (fichiers sains et sans guillemets parasites).")


In [ ]:
import pandas as pd
import csv
from pathlib import Path

# === Fonction pour détecter automatiquement le séparateur ===
def detect_separator(path, sample_size=4096):
    """Détecte le séparateur le plus probable dans un fichier CSV."""
    try:
        with open(path, "r", encoding="utf-8", errors="ignore") as f:
            sample = f.read(sample_size)
        sniffer = csv.Sniffer()
        sep = sniffer.sniff(sample).delimiter
        if sep not in [",", ";", "\t", "|"]:
            sep = ";"
    except Exception:
        sep = ";"
    return sep


# === Fonction principale de visualisation ===
def check_all_clean_files(folder="data/clean", nrows=5, max_cols=10):
    """Affiche un aperçu des fichiers CSV nettoyés (séparateur, colonnes, aperçu des données)."""
    clean_dir = Path(folder)
    clean_files = list(clean_dir.glob("*_clean.csv"))

    if not clean_files:
        raise FileNotFoundError(f"⚠️ Aucun fichier '_clean.csv' trouvé dans {folder}/")

    print(f"📂 {len(clean_files)} fichiers trouvés dans {folder}/\n")

    for f in clean_files:
        sep = detect_separator(f)
        print(f"=== {f.name} ===")
        print(f"   🔹 Séparateur détecté : '{sep}'")

        try:
            # Lecture de l'en-tête pour connaître le nombre total de colonnes
            header = pd.read_csv(f, sep=sep, nrows=0, encoding="utf-8", low_memory=False)
            nb_cols = len(header.columns)

            # Lecture partielle : jusqu’à 10 colonnes maximum, ou moins si le fichier en a moins
            cols_to_read = list(range(min(max_cols, nb_cols)))
            df = pd.read_csv(f, sep=sep, nrows=nrows, usecols=cols_to_read, encoding="utf-8", low_memory=False)

            print(f"   🔹 Colonnes affichées : {len(df.columns)} sur {nb_cols} totales")
            display(df)
        except Exception as e:
            print(f"⚠️ Erreur de lecture sur {f.name}: {e}")
        print("-" * 120)


# === Appel de la fonction ===
check_all_clean_files("data/clean")


In [ ]:
# === ETAPE 5 : TEST DU NETTOYAGE DES BASES DE DONNÉES ===
# Objectif : vérifier la cohérence, la propreté et la structure de chaque base clean

import pandas as pd
import numpy as np
import unicodedata
import csv
from pathlib import Path
import os

# --- Dossier contenant les bases nettoyées
CLEAN_DIR = Path("data/clean")

# --- Liste des fichiers à vérifier
files = sorted(CLEAN_DIR.glob("*.csv"))
print(f"📂 {len(files)} fichiers détectés dans {CLEAN_DIR}\n")

# --- Fonctions utilitaires
def normalize_text(s):
    if pd.isna(s):
        return np.nan
    s = str(s).strip().upper()
    s = "".join(c for c in unicodedata.normalize("NFKD", s) if not unicodedata.combining(c))
    s = s.replace("-", " ").replace("'", " ")
    return " ".join(s.split())

def detect_separator(path):
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        sample = f.read(4096)
    try:
        sep = csv.Sniffer().sniff(sample).delimiter
    except Exception:
        sep = ";"
    return sep

def test_clean_file(path):
    print(f"=== {path.name} ===")
    sep = detect_separator(path)
    try:
        df = pd.read_csv(path, sep=sep, encoding="utf-8", low_memory=False)
    except Exception as e:
        print(f"⚠️ Erreur de lecture : {e}\n")
        return None

    print(f"🔹 Lignes : {len(df)}, Colonnes : {len(df.columns)}, Séparateur : '{sep}'")

    # 1️⃣ Harmonisation de format
    if "code_commune" in df.columns:
        df["code_commune"] = df["code_commune"].astype(str).str.extract(r"(\d+)")[0].str.zfill(5)
    if "commune" in df.columns:
        df["commune"] = df["commune"].apply(normalize_text)

    # 2️⃣ Doublons
    dups = df.duplicated().sum()
    print(f"   🔁 Doublons détectés : {dups}")

    # 3️⃣ Valeurs manquantes
    missing = df.isna().mean().round(2)
    top_missing = missing[missing > 0].sort_values(ascending=False).head(5)
    if not top_missing.empty:
        print("   ⚠️ Colonnes avec NaN :", dict(top_missing))
    else:
        print("   ✅ Aucune valeur manquante significative")

    # 4️⃣ Uniformisation noms colonnes
    normalized_cols = [unicodedata.normalize("NFKD", c).encode("ascii", "ignore").decode("utf-8").lower().replace(" ", "_") for c in df.columns]
    if df.columns.tolist() != normalized_cols:
        print("   ⚙️ Correction potentielle des noms de colonnes incohérents")

    # 5️⃣ Types de données
    print("   📊 Types détectés :")
    print(df.dtypes.head())

    # 6️⃣ Normalisation du texte
    if "commune" in df.columns:
        uniques = df["commune"].nunique()
        print(f"   🧾 Communes uniques : {uniques}")

    # 7️⃣ Caractères parasites
    example_str = df.select_dtypes(include="object").astype(str).apply(lambda x: x.str.contains("[€,$,\t,;]", regex=True)).any()
    if example_str.any():
        print("   ⚠️ Caractères parasites détectés dans certaines colonnes texte")

    # 8️⃣ Clés communes
    if "code_commune" in df.columns:
        valid_keys = df["code_commune"].notna().sum()
        print(f"   🧩 Clé 'code_commune' présente ({valid_keys} valeurs valides)")

    # 9️⃣ Contrôle de cohérence simple (ex : surface > 0)
    if "surface_reelle_bati" in df.columns:
        negatives = (df["surface_reelle_bati"] <= 0).sum()
        if negatives > 0:
            print(f"   ⚠️ {negatives} valeurs de surface non valides (≤0)")
        else:
            print("   ✅ Toutes les surfaces sont positives")

    if set(["valeur_fonciere", "surface_reelle_bati"]).issubset(df.columns):
        df["prix_m2"] = df["valeur_fonciere"] / df["surface_reelle_bati"]
        mean_price = df["prix_m2"].mean(skipna=True)
        print(f"   💶 Prix moyen estimé au m² : {round(mean_price,2)}")

    # 10️⃣ Aperçu visuel
    print("\n   🧾 Aperçu :")
    display(df.head(5).iloc[:, :min(8, len(df.columns))])
    print("-" * 120)
    return df

# --- Lancement du test sur toutes les bases clean
for f in files:
    test_clean_file(f)


In [ ]:
# === ETAPE 1 : DÉTECTION DES ERREURS DE FORMAT ===
# Objectif : détecter les problèmes de lecture, encodage ou séparateur sur toutes les bases clean

import pandas as pd
import csv
from pathlib import Path

# --- Dossier contenant les bases clean
CLEAN_DIR = Path("data/clean")

# --- Recherche de tous les fichiers .csv dans le dossier
files = sorted(CLEAN_DIR.glob("*.csv"))
print(f"📂 {len(files)} fichiers détectés dans {CLEAN_DIR}\n")

# --- Fonction de test de format
def detect_format_issues(path):
    report = {"fichier": path.name, "ok": True, "erreur": None, "colonnes": 0, "lignes": 0, "sep": None}

    try:
        # Détection automatique du séparateur
        with open(path, "r", encoding="utf-8", errors="ignore") as f:
            sample = f.read(4096)
        try:
            sep = csv.Sniffer().sniff(sample).delimiter
        except Exception:
            sep = ";"
        report["sep"] = sep

        # Lecture test sur 100 premières lignes
        df = pd.read_csv(path, sep=sep, encoding="utf-8", nrows=100, low_memory=False)
        report["colonnes"] = df.shape[1]
        report["lignes"] = len(df)

        # Test de cohérence de structure
        if df.shape[1] < 3:
            report["ok"] = False
            report["erreur"] = "Trop peu de colonnes (mauvais séparateur ou structure)"
        elif df.shape[0] == 0:
            report["ok"] = False
            report["erreur"] = "Fichier vide ou mal encodé"

    except Exception as e:
        report["ok"] = False
        report["erreur"] = str(e)

    return report

# --- Lancement du test pour toutes les bases
results = []
for f in files:
    print(f"🔎 Vérification de {f.name} ...")
    res = detect_format_issues(f)
    results.append(res)
    if not res["ok"]:
        print(f"   ⚠️ Erreur détectée : {res['erreur']}")
    else:
        print(f"   ✅ OK ({res['colonnes']} colonnes, {res['lignes']} lignes, sep='{res['sep']}')")
    print("-" * 100)

# --- Résumé global
df_report = pd.DataFrame(results)
print("\n📊 === RÉSUMÉ DES ERREURS DE FORMAT ===")
display(df_report)

# --- Statistiques générales
total = len(df_report)
ok = df_report["ok"].sum()
ko = total - ok

print(f"✅ Fichiers valides : {ok}/{total}")
print(f"⚠️ Fichiers avec erreurs : {ko}/{total}")

if ko > 0:
    print("\n🧾 Liste des fichiers problématiques :")
    display(df_report.loc[df_report["ok"] == False, ["fichier", "erreur"]])


In [ ]:
# === NETTOYAGE GLOBAL + SUPPRESSION DES DOUBLONS (EN PLACE) ===
# Objectif :
# 1️⃣ Supprimer les fichiers temporaires dans data/clean/
# 2️⃣ Supprimer les doublons dans tous les fichiers clean et réécrire directement le fichier
# 3️⃣ Fournir un rapport complet et propre

import pandas as pd
import csv
from pathlib import Path

# --- Dossier contenant les bases nettoyées
CLEAN_DIR = Path("data/clean")

print("🔍 Étape 1 : Nettoyage du dossier 'data/clean'...\n")

# --- Étape 1 : supprimer les fichiers temporaires
suffixes_a_supprimer = ["_nodup.csv", "_fixed.csv", "_clean_fixed.csv"]

suppr = 0
for f in CLEAN_DIR.glob("*.csv"):
    if any(suffix in f.name for suffix in suffixes_a_supprimer):
        f.unlink()
        suppr += 1

print(f"🧹 {suppr} fichiers temporaires supprimés.")
print("✅ Dossier 'data/clean' revenu à l'état initial (seuls les *_clean.csv sont conservés).\n")

# --- Étape 2 : suppression des doublons dans chaque fichier
print("⚙️ Étape 2 : Suppression des doublons (fichiers réécrits en place)...\n")

def detect_separator(path):
    """Détection automatique du séparateur CSV probable."""
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        sample = f.read(4096)
    try:
        sep = csv.Sniffer().sniff(sample).delimiter
    except Exception:
        sep = ";"
    return sep

def clean_duplicates_in_place(path):
    """Supprime les doublons et réécrit le fichier directement."""
    try:
        sep = detect_separator(path)
        df = pd.read_csv(path, sep=sep, encoding="utf-8", low_memory=False)
    except Exception as e:
        print(f"⚠️ Erreur de lecture sur {path.name} : {e}")
        return None

    before = len(df)
    df = df.drop_duplicates()
    after = len(df)
    removed = before - after
    pct = round((removed / before) * 100, 2) if before > 0 else 0

    # Réécriture directe du fichier
    df.to_csv(path, index=False, encoding="utf-8", sep=",")

    print(f"✅ {path.name} : {removed} doublons supprimés ({pct}%) — Fichier mis à jour")
    return {"fichier": path.name, "lignes_avant": before, "lignes_apres": after, "doublons_supprimes": removed, "pct": pct}

# --- Application du correctif
files = sorted(CLEAN_DIR.glob("*.csv"))
print(f"📂 {len(files)} fichiers détectés dans {CLEAN_DIR}\n")

results = []
for f in files:
    res = clean_duplicates_in_place(f)
    if res:
        results.append(res)

# --- Étape 3 : Synthèse globale
print("\n📊 === SYNTHÈSE FINALE DU NETTOYAGE ===")
df_res = pd.DataFrame(results)
display(df_res)

total_removed = df_res["doublons_supprimes"].sum()
print(f"\n🧾 Total de doublons supprimés : {total_removed}")
print("🎯 Tous les fichiers ont été nettoyés et mis à jour sans duplication.")
print("✅ Tu peux maintenant relancer ton test de nettoyage pour confirmer qu'il n'y a plus de doublons.")


In [ ]:
# === ETAPE 2 : DÉTECTION DES DOUBLONS ===
# Objectif : repérer les lignes répétées dans toutes les bases clean

import pandas as pd
from pathlib import Path
import csv

# --- Dossier contenant les bases nettoyées
CLEAN_DIR = Path("data/clean")

# --- Recherche de tous les fichiers CSV
files = sorted(CLEAN_DIR.glob("*.csv"))
print(f"📂 {len(files)} fichiers détectés dans {CLEAN_DIR}\n")

def detect_separator(path):
    """Détecte automatiquement le séparateur probable (, ou ;)"""
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        sample = f.read(4096)
    try:
        sep = csv.Sniffer().sniff(sample).delimiter
    except Exception:
        sep = ";"
    return sep

def check_duplicates(path):
    """Analyse un fichier CSV et détecte les doublons."""
    try:
        sep = detect_separator(path)
        df = pd.read_csv(path, sep=sep, encoding="utf-8", low_memory=False)
    except Exception as e:
        print(f"⚠️ Erreur de lecture sur {path.name} : {e}")
        return {"fichier": path.name, "ok": False, "erreur": str(e), "doublons": None}

    # --- Détection des doublons
    nb_total = len(df)
    nb_doublons = df.duplicated().sum()
    pct = round((nb_doublons / nb_total) * 100, 2) if nb_total > 0 else 0

    print(f"=== {path.name} ===")
    print(f"🔹 Lignes totales : {nb_total}")
    print(f"🔁 Doublons détectés : {nb_doublons} ({pct} %)")
    
    if nb_doublons > 0:
        print("🧾 Exemple de doublons :")
        display(df[df.duplicated()].head(5))
    else:
        print("✅ Aucun doublon trouvé")
    
    print("-" * 100)
    return {"fichier": path.name, "ok": True, "lignes": nb_total, "doublons": nb_doublons, "pourcentage": pct}

# --- Analyse de tous les fichiers
results = []
for f in files:
    res = check_duplicates(f)
    results.append(res)

# --- Synthèse globale
df_doublons = pd.DataFrame(results)
print("\n📊 === SYNTHÈSE DES DOUBLONS ===")
display(df_doublons)

# --- Statistiques globales
total = len(df_doublons)
ok_files = (df_doublons["doublons"] == 0).sum()
print(f"✅ Fichiers sans doublon : {ok_files}/{total}")
print(f"⚠️ Fichiers contenant des doublons : {total - ok_files}/{total}")


In [ ]:
# === CORRECTIF ÉTAPE 3 : SUPPRESSION DES COLONNES VIDES OU TROP INCOMPLÈTES ===
# Objectif :
# - Supprimer les colonnes 100% NaN ou avec plus de 50% de valeurs manquantes
# - Réécrire les fichiers clean existants (pas de duplication)
# - Fournir un rapport clair de la réduction des colonnes

import pandas as pd
import csv
from pathlib import Path

# --- Dossier contenant les fichiers nettoyés
CLEAN_DIR = Path("data/clean")
files = sorted(CLEAN_DIR.glob("*.csv"))
print(f"📂 {len(files)} fichiers détectés dans {CLEAN_DIR}\n")

# --- Fonction utilitaire pour détecter le séparateur
def detect_separator(path):
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        sample = f.read(4096)
    try:
        sep = csv.Sniffer().sniff(sample).delimiter
    except Exception:
        sep = ";"
    return sep

# --- Nettoyage et réécriture en place
def clean_missing_columns_in_place(path, seuil=0.5):
    """Supprime les colonnes vides ou avec plus de `seuil` de NaN."""
    try:
        sep = detect_separator(path)
        df = pd.read_csv(path, sep=sep, encoding="utf-8", low_memory=False)
    except Exception as e:
        print(f"⚠️ Erreur de lecture sur {path.name} : {e}")
        return None

    print(f"=== {path.name} ===")
    before_cols = len(df.columns)
    before_rows = len(df)

    # --- Calcul du taux de valeurs manquantes
    missing_ratio = df.isna().mean()

    # --- Colonnes à supprimer
    empty_cols = list(missing_ratio[missing_ratio == 1.0].index)
    incomplete_cols = list(missing_ratio[missing_ratio > seuil].index)
    to_drop = set(empty_cols + incomplete_cols)

    # --- Suppression
    df = df.drop(columns=to_drop, errors="ignore")

    after_cols = len(df.columns)
    removed_cols = before_cols - after_cols
    pct_removed = round((removed_cols / before_cols) * 100, 2) if before_cols > 0 else 0

    # --- Réécriture du fichier propre
    df.to_csv(path, index=False, encoding="utf-8", sep=",")
    print(f"✅ Colonnes supprimées : {removed_cols} ({pct_removed}%)")
    if removed_cols > 0:
        print(f"   🧹 {list(to_drop)[:10]}{' ...' if len(to_drop) > 10 else ''}")
    else:
        print("   ✅ Aucune colonne supprimée")
    print(f"   🔸 Fichier mis à jour : {path.name} ({after_cols} colonnes conservées, {before_rows} lignes)")
    print("-" * 100)

    return {
        "fichier": path.name,
        "colonnes_avant": before_cols,
        "colonnes_apres": after_cols,
        "supprimées": removed_cols,
        "pct_supprimées": pct_removed
    }

# --- Application à toutes les bases clean
results = []
for f in files:
    res = clean_missing_columns_in_place(f, seuil=0.5)
    if res:
        results.append(res)

# --- Synthèse globale
df_summary = pd.DataFrame(results)
print("\n📊 === SYNTHÈSE DU NETTOYAGE DES COLONNES ===")
display(df_summary)

total_removed = df_summary["supprimées"].sum()
print(f"🧾 Total de colonnes supprimées : {total_removed}")
print("🎯 Tous les fichiers ont été mis à jour directement sans duplication.")


In [ ]:
# === ETAPE 3 : ANALYSE DES VALEURS MANQUANTES ===
# Objectif :
# - Identifier les colonnes comportant des NaN ou cellules vides
# - Afficher des statistiques claires pour chaque base
# - Ne pas générer de fichier sur le disque

import pandas as pd
import csv
from pathlib import Path

# --- Dossier contenant les fichiers nettoyés (déjà sans doublons)
CLEAN_DIR = Path("data/clean")
files = sorted(CLEAN_DIR.glob("*.csv"))
print(f"📂 {len(files)} fichiers détectés dans {CLEAN_DIR}\n")

def detect_separator(path):
    """Détecte automatiquement le séparateur probable (, ou ;)"""
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        sample = f.read(4096)
    try:
        sep = csv.Sniffer().sniff(sample).delimiter
    except Exception:
        sep = ";"
    return sep

def analyze_missing_values(path):
    """Analyse le taux de valeurs manquantes d'un fichier."""
    try:
        sep = detect_separator(path)
        df = pd.read_csv(path, sep=sep, encoding="utf-8", low_memory=False)
    except Exception as e:
        print(f"⚠️ Erreur de lecture sur {path.name} : {e}")
        return {"fichier": path.name, "ok": False, "erreur": str(e)}

    print(f"=== {path.name} ===")
    print(f"🔹 {len(df)} lignes, {len(df.columns)} colonnes")

    # --- Calcul des NaN par colonne
    missing_ratio = df.isna().mean().round(3)
    missing_ratio = missing_ratio[missing_ratio > 0].sort_values(ascending=False)

    # --- Statistiques globales
    nb_col_nan = len(missing_ratio)
    taux_moyen_nan = round(missing_ratio.mean() * 100, 2) if nb_col_nan > 0 else 0
    taux_median_nan = round(missing_ratio.median() * 100, 2) if nb_col_nan > 0 else 0

    if nb_col_nan > 0:
        print(f"⚠️ {nb_col_nan} colonnes avec des valeurs manquantes ({taux_moyen_nan}% en moyenne)")
        print("🔝 Top 10 des colonnes les plus incomplètes :")
        display(missing_ratio.head(10))
    else:
        print("✅ Aucune valeur manquante détectée dans ce fichier.")
    
    print("-" * 100)
    return {
        "fichier": path.name,
        "colonnes": len(df.columns),
        "lignes": len(df),
        "nb_colonnes_nan": nb_col_nan,
        "taux_moyen_nan(%)": taux_moyen_nan,
        "taux_median_nan(%)": taux_median_nan,
    }

# --- Application à toutes les bases
results = []
for f in files:
    res = analyze_missing_values(f)
    results.append(res)

# --- Synthèse globale
df_nan = pd.DataFrame(results)
print("\n📊 === SYNTHÈSE GLOBALE DES VALEURS MANQUANTES ===")
display(df_nan)

# --- Indicateurs globaux
nb_total = len(df_nan)
nb_sans_nan = (df_nan["nb_colonnes_nan"] == 0).sum()
nb_avec_nan = nb_total - nb_sans_nan
taux_moyen_global = round(df_nan["taux_moyen_nan(%)"].mean(), 2)

print(f"✅ Bases sans NaN : {nb_sans_nan}/{nb_total}")
print(f"⚠️ Bases avec NaN : {nb_avec_nan}/{nb_total}")
print(f"📈 Taux moyen global de NaN sur l'ensemble des fichiers : {taux_moyen_global}%")


In [ ]:
# === ETAPE 4 : UNIFORMISATION DES NOMS DE COLONNES ===
# Objectif :
#   - Vérifier et corriger les noms de colonnes (minuscules, sans accents, underscores)
#   - Supprimer les caractères spéciaux et doublons
#   - Réécrire directement les fichiers nettoyés
#   - Fournir un résumé clair avant/après

import pandas as pd
import unicodedata
import csv
from pathlib import Path

# --- Dossier contenant les fichiers nettoyés
CLEAN_DIR = Path("data/clean")
files = sorted(CLEAN_DIR.glob("*.csv"))
print(f"📂 {len(files)} fichiers détectés dans {CLEAN_DIR}\n")

def detect_separator(path):
    """Détection automatique du séparateur probable (, ou ;)"""
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        sample = f.read(4096)
    try:
        sep = csv.Sniffer().sniff(sample).delimiter
    except Exception:
        sep = ";"
    return sep

def normalize_column_name(name):
    """Normalise un nom de colonne : minuscules, pas d'accents, underscores, caractères alphanumériques uniquement."""
    name = str(name).strip().lower()
    name = "".join(c for c in unicodedata.normalize("NFKD", name) if not unicodedata.combining(c))
    name = name.replace(" ", "_").replace("-", "_").replace("/", "_").replace(".", "_")
    name = "".join(ch for ch in name if ch.isalnum() or ch == "_")
    name = "_".join([tok for tok in name.split("_") if tok])  # supprime les underscores multiples
    return name

def clean_column_names_in_place(path):
    """Uniformise les noms de colonnes et réécrit le fichier en place."""
    try:
        sep = detect_separator(path)
        df = pd.read_csv(path, sep=sep, encoding="utf-8", low_memory=False)
    except Exception as e:
        print(f"⚠️ Erreur de lecture sur {path.name} : {e}")
        return None

    print(f"=== {path.name} ===")
    before_cols = df.columns.tolist()

    # --- Normalisation des noms de colonnes
    normalized_cols = [normalize_column_name(c) for c in before_cols]
    df.columns = normalized_cols

    # --- Gestion des doublons éventuels
    if len(set(df.columns)) < len(df.columns):
        seen = {}
        new_cols = []
        for c in df.columns:
            if c not in seen:
                seen[c] = 1
                new_cols.append(c)
            else:
                seen[c] += 1
                new_cols.append(f"{c}_{seen[c]}")
        df.columns = new_cols
        print("⚠️ Doublons de colonnes détectés et corrigés automatiquement.")

    # --- Sauvegarde du fichier corrigé
    df.to_csv(path, index=False, encoding="utf-8", sep=",")

    # --- Résumé
    after_cols = df.columns.tolist()
    changes = {b: a for b, a in zip(before_cols, after_cols) if b != a}
    print(f"✅ Colonnes renommées : {len(changes)} / {len(before_cols)}")
    if len(changes) > 0:
        print(f"   🧾 Exemples : {list(changes.items())[:5]}{' ...' if len(changes) > 5 else ''}")
    print(f"   🔸 Fichier mis à jour : {path.name} ({len(df.columns)} colonnes)")
    print("-" * 100)

    return {
        "fichier": path.name,
        "colonnes_totales": len(df.columns),
        "colonnes_renommees": len(changes),
        "pct_renommees": round((len(changes) / len(df.columns)) * 100, 2) if len(df.columns) > 0 else 0
    }

# --- Application à toutes les bases clean
results = []
for f in files:
    res = clean_column_names_in_place(f)
    if res:
        results.append(res)

# --- Synthèse globale
df_summary = pd.DataFrame(results)
print("\n📊 === SYNTHÈSE DU NETTOYAGE DES NOMS DE COLONNES ===")
display(df_summary)

total_renamed = df_summary["colonnes_renommees"].sum()
print(f"🧾 Total de colonnes renommées : {total_renamed}")
print("🎯 Tous les fichiers ont été mis à jour directement sans duplication.")


In [ ]:
# === ETAPE 5 (corrigée) : CONTRÔLE ET CORRECTION DES TYPES DE DONNÉES ===
# Correction : suppression de 'infer_datetime_format' et suppression des warnings répétitifs

import pandas as pd
import numpy as np
import csv
import warnings
from pathlib import Path

# Désactivation des warnings inutiles de pandas
warnings.filterwarnings("ignore", category=UserWarning, module="pandas")

# --- Dossier contenant les fichiers clean
CLEAN_DIR = Path("data/clean")
files = sorted(CLEAN_DIR.glob("*.csv"))
print(f"📂 {len(files)} fichiers détectés dans {CLEAN_DIR}\n")

def detect_separator(path):
    """Détecte automatiquement le séparateur probable (, ou ;)"""
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        sample = f.read(4096)
    try:
        sep = csv.Sniffer().sniff(sample).delimiter
    except Exception:
        sep = ";"
    return sep

def infer_and_fix_types_in_place(path):
    """Analyse et corrige les types de données d’un fichier."""
    try:
        sep = detect_separator(path)
        df = pd.read_csv(path, sep=sep, encoding="utf-8", low_memory=False)
    except Exception as e:
        print(f"⚠️ Erreur de lecture sur {path.name} : {e}")
        return None

    print(f"=== {path.name} ===")
    before_types = df.dtypes.copy()
    conversions = {}

    for col in df.columns:
        series = df[col]

        # Tentative de conversion des chaînes vers des types utiles
        if series.dtype == "object":
            # 1️⃣ Conversion numérique
            num_series = pd.to_numeric(series.astype(str).str.replace(",", ".").str.replace(" ", ""), errors="coerce")
            ratio_num = num_series.notna().mean()
            if ratio_num > 0.9:  # au moins 90 % de valeurs numériques
                df[col] = num_series
                conversions[col] = "float"
                continue

            # 2️⃣ Conversion date (version corrigée)
            try:
                date_series = pd.to_datetime(series, errors="coerce")
                ratio_date = date_series.notna().mean()
                if ratio_date > 0.9:
                    df[col] = date_series
                    conversions[col] = "datetime"
                    continue
            except Exception:
                pass

        # 3️⃣ Conversion float → Int64 quand possible
        if df[col].dtype == "float64" and df[col].dropna().apply(float.is_integer).all():
            df[col] = df[col].astype("Int64")
            conversions[col] = "Int64"

    # Sauvegarde du fichier corrigé
    df.to_csv(path, index=False, encoding="utf-8", sep=",")

    after_types = df.dtypes
    changed = {col: (before_types[col], after_types[col]) for col in df.columns if before_types[col] != after_types[col]}

    print(f"✅ Colonnes converties : {len(conversions)}")
    if len(conversions) > 0:
        print(f"   🧾 Exemples : {list(conversions.items())[:5]}{' ...' if len(conversions) > 5 else ''}")
    print(f"   🔸 Fichier mis à jour : {path.name}")
    print("-" * 100)

    return {
        "fichier": path.name,
        "colonnes_totales": len(df.columns),
        "colonnes_converties": len(conversions),
        "types_changés": len(changed)
    }

# --- Application à toutes les bases clean
results = []
for f in files:
    res = infer_and_fix_types_in_place(f)
    if res:
        results.append(res)

# --- Synthèse globale
df_summary = pd.DataFrame(results)
print("\n📊 === SYNTHÈSE DU CONTRÔLE DES TYPES ===")
display(df_summary)

total_converted = df_summary["colonnes_converties"].sum()
print(f"🧾 Total de colonnes converties : {total_converted}")
print("🎯 Tous les fichiers ont été vérifiés et mis à jour sans duplication.")


In [ ]:
# === ETAPE 6 : NORMALISATION DES TEXTES ===
# Objectif :
#   - Nettoyer les colonnes texte pour supprimer les accents, symboles et espaces parasites
#   - Uniformiser la casse (majuscule cohérente)
#   - Réécrire les fichiers directement
#   - Fournir un résumé clair du nettoyage

import pandas as pd
import unicodedata
import re
import csv
from pathlib import Path

# --- Dossier contenant les fichiers nettoyés
CLEAN_DIR = Path("data/clean")
files = sorted(CLEAN_DIR.glob("*.csv"))
print(f"📂 {len(files)} fichiers détectés dans {CLEAN_DIR}\n")

def detect_separator(path):
    """Détecte automatiquement le séparateur (, ou ;)"""
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        sample = f.read(4096)
    try:
        sep = csv.Sniffer().sniff(sample).delimiter
    except Exception:
        sep = ";"
    return sep

def normalize_text(value):
    """Nettoie et normalise les chaînes de texte."""
    if pd.isna(value):
        return value
    value = str(value).strip()

    # Supprimer les accents
    value = "".join(c for c in unicodedata.normalize("NFKD", value) if not unicodedata.combining(c))

    # Supprimer les symboles et caractères spéciaux
    value = re.sub(r"[€%$;'\t\n\r]", " ", value)

    # Remplacer tirets et apostrophes par espace
    value = re.sub(r"[-']", " ", value)

    # Supprimer les espaces multiples
    value = re.sub(r"\s+", " ", value)

    # Mise en majuscule pour les textes courts (ex: communes, catégories)
    if len(value) <= 40:
        value = value.upper()
    else:
        value = value.capitalize()

    return value.strip()

def normalize_text_columns_in_place(path):
    """Normalise les colonnes texte d’un fichier CSV et réécrit le fichier."""
    try:
        sep = detect_separator(path)
        df = pd.read_csv(path, sep=sep, encoding="utf-8", low_memory=False)
    except Exception as e:
        print(f"⚠️ Erreur de lecture sur {path.name} : {e}")
        return None

    print(f"=== {path.name} ===")
    before_cols = df.shape[1]
    before_rows = df.shape[0]

    # Colonnes texte à traiter
    text_cols = df.select_dtypes(include=["object"]).columns
    print(f"🧾 Colonnes texte détectées : {len(text_cols)} / {before_cols}")

    if len(text_cols) > 0:
        for col in text_cols:
            df[col] = df[col].apply(normalize_text)

        # Réécriture directe du fichier
        df.to_csv(path, index=False, encoding="utf-8", sep=",")
        print(f"✅ Colonnes texte normalisées ({len(text_cols)} colonnes traitées)")
    else:
        print("ℹ️ Aucune colonne texte à normaliser.")
    print(f"   🔸 Fichier mis à jour : {path.name} ({before_rows} lignes)")
    print("-" * 100)

    return {
        "fichier": path.name,
        "colonnes_totales": before_cols,
        "colonnes_texte": len(text_cols)
    }

# --- Application à toutes les bases clean
results = []
for f in files:
    res = normalize_text_columns_in_place(f)
    if res:
        results.append(res)

# --- Synthèse globale
df_summary = pd.DataFrame(results)
print("\n📊 === SYNTHÈSE DU NETTOYAGE TEXTUEL ===")
display(df_summary)

total_text_cols = df_summary["colonnes_texte"].sum()
print(f"🧾 Total de colonnes texte normalisées : {total_text_cols}")
print("🎯 Tous les fichiers ont été normalisés et mis à jour sans duplication.")


In [ ]:
# === ETAPE 7 : SUPPRESSION DES CARACTÈRES PARASITES ===
# Objectif :
#   - Supprimer les caractères invisibles ou non imprimables dans les champs texte
#   - Nettoyer les guillemets, tabulations, espaces spéciaux (\xa0, \t, etc.)
#   - Réécrire les fichiers directement
#   - Fournir un rapport clair de nettoyage

import pandas as pd
import csv
import re
from pathlib import Path

# --- Dossier contenant les fichiers nettoyés
CLEAN_DIR = Path("data/clean")
files = sorted(CLEAN_DIR.glob("*.csv"))
print(f"📂 {len(files)} fichiers détectés dans {CLEAN_DIR}\n")

def detect_separator(path):
    """Détecte automatiquement le séparateur probable (, ou ;)"""
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        sample = f.read(4096)
    try:
        sep = csv.Sniffer().sniff(sample).delimiter
    except Exception:
        sep = ";"
    return sep

def clean_parasites_in_place(path):
    """Supprime les caractères parasites dans les colonnes texte et réécrit le fichier."""
    try:
        sep = detect_separator(path)
        df = pd.read_csv(path, sep=sep, encoding="utf-8", low_memory=False)
    except Exception as e:
        print(f"⚠️ Erreur de lecture sur {path.name} : {e}")
        return None

    print(f"=== {path.name} ===")
    before_cols = df.shape[1]
    before_rows = df.shape[0]

    text_cols = df.select_dtypes(include=["object"]).columns
    print(f"🧾 Colonnes texte détectées : {len(text_cols)} / {before_cols}")

    # --- Fonction de nettoyage cellule
    def clean_cell(value):
        if pd.isna(value):
            return value
        value = str(value)

        # Supprime les espaces non standard et caractères de contrôle
        value = re.sub(r"[\x00-\x1F\x7F\xa0\u200b\r\t]", " ", value)

        # Supprime guillemets et doubles séparateurs
        value = value.replace('"', "").replace("'", "")

        # Supprime les espaces multiples
        value = re.sub(r"\s+", " ", value)

        return value.strip()

    # --- Application sur toutes les colonnes texte
    if len(text_cols) > 0:
        for col in text_cols:
            df[col] = df[col].apply(clean_cell)

        # Sauvegarde directe du fichier propre
        df.to_csv(path, index=False, encoding="utf-8", sep=",")
        print(f"✅ Colonnes nettoyées : {len(text_cols)} (fichier réécrit)")
    else:
        print("ℹ️ Aucun champ texte à nettoyer.")
    print(f"   🔸 Fichier mis à jour : {path.name} ({before_rows} lignes)")
    print("-" * 100)

    return {
        "fichier": path.name,
        "colonnes_totales": before_cols,
        "colonnes_texte": len(text_cols)
    }

# --- Application du nettoyage à toutes les bases
results = []
for f in files:
    res = clean_parasites_in_place(f)
    if res:
        results.append(res)

# --- Synthèse globale
df_summary = pd.DataFrame(results)
print("\n📊 === SYNTHÈSE DU NETTOYAGE DES CARACTÈRES PARASITES ===")
display(df_summary)

total_cleaned = df_summary["colonnes_texte"].sum()
print(f"🧾 Total de colonnes texte nettoyées : {total_cleaned}")
print("🎯 Tous les fichiers ont été nettoyés et mis à jour sans duplication.")


In [ ]:
# === ETAPE 8 : CRÉATION ET VÉRIFICATION DES CLÉS COMMUNES ===
# Objectif :
#   - Créer / uniformiser les colonnes code_commune et commune_std
#   - Normaliser les noms de communes
#   - Vérifier la cohérence et réécrire les fichiers directement

import pandas as pd
import numpy as np
import unicodedata
import re
import csv
from pathlib import Path

# --- Dossier contenant les fichiers nettoyés
CLEAN_DIR = Path("data/clean")
files = sorted(CLEAN_DIR.glob("*.csv"))
print(f"📂 {len(files)} fichiers détectés dans {CLEAN_DIR}\n")

# --- Utilitaires de normalisation
def detect_separator(path):
    """Détecte automatiquement le séparateur probable (, ou ;)"""
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        sample = f.read(4096)
    try:
        sep = csv.Sniffer().sniff(sample).delimiter
    except Exception:
        sep = ";"
    return sep

def normalize_commune_name(name):
    """Nettoie et standardise le nom de la commune."""
    if pd.isna(name):
        return np.nan
    name = str(name).strip().upper()
    name = "".join(c for c in unicodedata.normalize("NFKD", name) if not unicodedata.combining(c))
    name = re.sub(r"[-']", " ", name)
    name = re.sub(r"\s+", " ", name)
    return name.strip()

def extract_code_commune(value):
    """Extrait un code INSEE sur 5 chiffres si possible."""
    if pd.isna(value):
        return np.nan
    value = str(value)
    match = re.search(r"\d{5}", value)
    return match.group(0) if match else np.nan

def ensure_commune_keys(path):
    """Ajoute / vérifie code_commune et commune_std puis réécrit le fichier."""
    try:
        sep = detect_separator(path)
        df = pd.read_csv(path, sep=sep, encoding="utf-8", low_memory=False)
    except Exception as e:
        print(f"⚠️ Erreur de lecture sur {path.name} : {e}")
        return None

    print(f"=== {path.name} ===")
    before_cols = len(df.columns)
    before_rows = len(df)

    # --- Détection de colonnes similaires existantes
    code_col = next((c for c in df.columns if "insee" in c or "code_commune" in c), None)
    name_col = next((c for c in df.columns if "commune" in c and "std" not in c), None)

    # --- Création / nettoyage du code_commune
    if code_col:
        df["code_commune"] = df[code_col].apply(extract_code_commune)
    else:
        df["code_commune"] = np.nan

    # --- Création / nettoyage du nom de commune
    if name_col:
        df["commune_std"] = df[name_col].apply(normalize_commune_name)
    else:
        df["commune_std"] = np.nan

    # --- Rapport
    valid_codes = df["code_commune"].notna().sum()
    valid_names = df["commune_std"].notna().sum()
    print(f"🧩 code_commune : {valid_codes} valeurs valides / {before_rows}")
    print(f"🏙️ commune_std : {valid_names} valeurs valides / {before_rows}")

    # --- Réécriture directe du fichier
    df.to_csv(path, index=False, encoding="utf-8", sep=",")
    print(f"✅ Fichier mis à jour : {path.name} ({len(df.columns)} colonnes, {before_rows} lignes)")
    print("-" * 100)

    return {
        "fichier": path.name,
        "code_commune_valide": valid_codes,
        "commune_std_valide": valid_names,
        "total_lignes": before_rows
    }

# --- Application à toutes les bases clean
results = []
for f in files:
    res = ensure_commune_keys(f)
    if res:
        results.append(res)

# --- Synthèse globale
df_summary = pd.DataFrame(results)
print("\n📊 === SYNTHÈSE DES CLÉS COMMUNES ===")
display(df_summary)

# --- Indicateurs globaux
nb_total = len(df_summary)
avg_code_rate = round((df_summary["code_commune_valide"] / df_summary["total_lignes"]).mean() * 100, 2)
avg_name_rate = round((df_summary["commune_std_valide"] / df_summary["total_lignes"]).mean() * 100, 2)

print(f"✅ Moyenne de complétude du code_commune : {avg_code_rate}%")
print(f"✅ Moyenne de complétude du commune_std : {avg_name_rate}%")
print("🎯 Tous les fichiers ont été mis à jour avec des clés de jointure cohérentes.")


In [ ]:
# === ETAPE 9 : CONTRÔLE ET COHÉRENCE DES DONNÉES ===
# Objectif :
#   - Vérifier la cohérence logique des valeurs numériques
#   - Supprimer les lignes incohérentes (surfaces <= 0, valeurs foncières négatives, etc.)
#   - Fournir un rapport clair et réécrire les fichiers en place

import pandas as pd
import numpy as np
import csv
from pathlib import Path

# --- Dossier contenant les fichiers nettoyés
CLEAN_DIR = Path("data/clean")
files = sorted(CLEAN_DIR.glob("*.csv"))
print(f"📂 {len(files)} fichiers détectés dans {CLEAN_DIR}\n")

def detect_separator(path):
    """Détecte automatiquement le séparateur probable (, ou ;)"""
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        sample = f.read(4096)
    try:
        sep = csv.Sniffer().sniff(sample).delimiter
    except Exception:
        sep = ";"
    return sep

def control_coherence_in_place(path):
    """Vérifie et corrige les incohérences de cohérence logique des données."""
    try:
        sep = detect_separator(path)
        df = pd.read_csv(path, sep=sep, encoding="utf-8", low_memory=False)
    except Exception as e:
        print(f"⚠️ Erreur de lecture sur {path.name} : {e}")
        return None

    print(f"=== {path.name} ===")
    before_rows = len(df)
    anomalies = {}

    # --- 1️⃣ Surfaces négatives ou nulles
    if "surface_reelle_bati" in df.columns:
        anomalies["surface<=0"] = (df["surface_reelle_bati"] <= 0).sum(skipna=True)
        df = df[df["surface_reelle_bati"] > 0]

    # --- 2️⃣ Valeurs foncières négatives ou nulles
    if "valeur_fonciere" in df.columns:
        anomalies["valeur<=0"] = (df["valeur_fonciere"] <= 0).sum(skipna=True)
        df = df[df["valeur_fonciere"] > 0]

    # --- 3️⃣ Prix au m² irréalistes (si applicable)
    if set(["valeur_fonciere", "surface_reelle_bati"]).issubset(df.columns):
        df["prix_m2"] = df["valeur_fonciere"] / df["surface_reelle_bati"]
        anomalies["prix_m2<500"] = (df["prix_m2"] < 500).sum(skipna=True)
        anomalies["prix_m2>50000"] = (df["prix_m2"] > 50000).sum(skipna=True)
        df = df[(df["prix_m2"] >= 500) & (df["prix_m2"] <= 50000)]

    # --- 4️⃣ Valeurs négatives générales sur d’autres indicateurs
    num_cols = df.select_dtypes(include=[np.number]).columns
    for col in num_cols:
        if col not in ["valeur_fonciere", "surface_reelle_bati", "prix_m2"]:
            negatives = (df[col] < 0).sum(skipna=True)
            if negatives > 0:
                anomalies[f"{col}<0"] = negatives
                df = df[df[col] >= 0]

    # --- Calcul des suppressions
    after_rows = len(df)
    removed = before_rows - after_rows
    pct_removed = round((removed / before_rows) * 100, 2) if before_rows > 0 else 0

    # --- Sauvegarde du fichier propre
    df.to_csv(path, index=False, encoding="utf-8", sep=",")
    print(f"✅ {removed} lignes incohérentes supprimées ({pct_removed}%)")
    if anomalies:
        print("📊 Détails anomalies détectées :", anomalies)
    print(f"   🔸 Fichier mis à jour : {path.name} ({after_rows} lignes restantes)")
    print("-" * 100)

    return {
        "fichier": path.name,
        "lignes_avant": before_rows,
        "lignes_apres": after_rows,
        "supprimees": removed,
        "pct_supprimees": pct_removed
    }

# --- Application à toutes les bases clean
results = []
for f in files:
    res = control_coherence_in_place(f)
    if res:
        results.append(res)

# --- Synthèse globale
df_summary = pd.DataFrame(results)
print("\n📊 === SYNTHÈSE DU CONTRÔLE DE COHÉRENCE ===")
display(df_summary)

total_removed = df_summary["supprimees"].sum()
print(f"🧾 Total de lignes incohérentes supprimées : {total_removed}")
print("🎯 Tous les fichiers ont été nettoyés et mis à jour sans duplication.")


In [ ]:
# === ETAPE 10 : VÉRIFICATION STRUCTURELLE ET FORMAT FINAL ===
# Objectif :
#   - Vérifier la structure, le format et la propreté finale de chaque fichier clean
#   - Contrôler encodage, séparateur, cohérence colonnes / lignes, taille et caractères parasites

import pandas as pd
import csv
import os
from pathlib import Path

# --- Dossier contenant les fichiers nettoyés
CLEAN_DIR = Path("data/clean")
files = sorted(CLEAN_DIR.glob("*.csv"))
print(f"📂 {len(files)} fichiers détectés dans {CLEAN_DIR}\n")

# --- Fonction de détection du séparateur
def detect_separator(path):
    """Détecte automatiquement le séparateur probable (, ou ;)"""
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        sample = f.read(4096)
    try:
        sep = csv.Sniffer().sniff(sample).delimiter
    except Exception:
        sep = ";"
    return sep

# --- Vérification structurelle
def verify_structure(path):
    report = {
        "fichier": path.name,
        "encodage": "UTF-8",
        "sep": None,
        "colonnes": 0,
        "lignes": 0,
        "caractères_parasites": False,
        "taille_ko": round(os.path.getsize(path) / 1024, 2),
        "statut": "✅ Conforme"
    }

    try:
        sep = detect_separator(path)
        report["sep"] = sep

        df = pd.read_csv(path, sep=sep, encoding="utf-8", low_memory=False)
        report["colonnes"] = len(df.columns)
        report["lignes"] = len(df)

        # Vérification des critères de base
        if report["colonnes"] < 5 or report["lignes"] < 10:
            report["statut"] = "⚠️ Structure suspecte (trop peu de données)"

        # Recherche de caractères parasites
        with open(path, "r", encoding="utf-8", errors="ignore") as f:
            content = f.read()
        if any(char in content for char in ["\x00", "\x1f", "\xa0", "\u200b", "; ;", ", ,"]):
            report["caractères_parasites"] = True
            report["statut"] = "⚠️ Caractères parasites détectés"

    except UnicodeDecodeError:
        report["encodage"] = "⚠️ Non UTF-8"
        report["statut"] = "⚠️ Encodage incorrect"
    except Exception as e:
        report["statut"] = f"❌ Erreur de lecture : {e}"

    return report

# --- Application de la vérification à toutes les bases
results = []
for f in files:
    res = verify_structure(f)
    results.append(res)
    print(f"🔎 Vérification {f.name} → {res['statut']} ({res['colonnes']} colonnes, {res['lignes']} lignes)")

# --- Synthèse globale
df_final = pd.DataFrame(results)
print("\n📊 === RAPPORT FINAL DE VALIDATION ===")
display(df_final)

# --- Statistiques globales
nb_total = len(df_final)
nb_valid = (df_final["statut"].str.contains("✅")).sum()
nb_warnings = (df_final["statut"].str.contains("⚠️")).sum()
nb_errors = (df_final["statut"].str.contains("❌")).sum()

print(f"\n✅ Fichiers conformes : {nb_valid}/{nb_total}")
print(f"⚠️ Fichiers à vérifier : {nb_warnings}/{nb_total}")
print(f"❌ Fichiers en erreur : {nb_errors}/{nb_total}")

if nb_errors == 0 and nb_warnings == 0:
    print("\n🎯 Tous les fichiers sont prêts pour l’analyse finale et la fusion inter-bases !")
else:
    print("\n📌 Certains fichiers nécessitent une vérification manuelle avant l’étape d’analyse.")


In [ ]:
# === RÉCAPITULATIF GLOBAL DES BASES APRÈS NETTOYAGE ===
# Objectif :
#   - Afficher nombre de lignes / colonnes par base
#   - Montrer un aperçu (5 lignes × 8 colonnes)
#   - Vérifier cohérence et propreté globale

import pandas as pd
import csv
from pathlib import Path

# --- Dossier contenant les fichiers nettoyés
CLEAN_DIR = Path("data/clean")
files = sorted(CLEAN_DIR.glob("*.csv"))
print(f"📂 {len(files)} fichiers détectés dans {CLEAN_DIR}\n")

def detect_separator(path):
    """Détecte automatiquement le séparateur probable (, ou ;)"""
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        sample = f.read(4096)
    try:
        sep = csv.Sniffer().sniff(sample).delimiter
    except Exception:
        sep = ";"
    return sep

# --- Lecture et résumé global
summaries = []

for path in files:
    try:
        sep = detect_separator(path)
        df = pd.read_csv(path, sep=sep, encoding="utf-8", low_memory=False)

        # Informations de base
        lignes, colonnes = df.shape
        nan_rate = round(df.isna().mean().mean() * 100, 2)
        numeric_cols = len(df.select_dtypes(include=["number"]).columns)
        text_cols = len(df.select_dtypes(include=["object"]).columns)

        summaries.append({
            "Fichier": path.name,
            "Lignes": lignes,
            "Colonnes": colonnes,
            "Colonnes numériques": numeric_cols,
            "Colonnes texte": text_cols,
            "Taux NaN (%)": nan_rate
        })

        print(f"=== {path.name} ===")
        print(f"🔹 {lignes} lignes | {colonnes} colonnes")
        print(f"   📊 {numeric_cols} numériques | {text_cols} texte | NaN moyen : {nan_rate}%")
        print("   🔸 Aperçu des premières colonnes :")
        display(df.head(5).iloc[:, :min(8, colonnes)])  # 5 lignes × 8 colonnes max
        print("-" * 120)

    except Exception as e:
        print(f"⚠️ Impossible de lire {path.name} : {e}")
        print("-" * 100)

# --- Synthèse finale
print("\n📊 === SYNTHÈSE GLOBALE DES BASES ===")
df_summary = pd.DataFrame(summaries)
display(df_summary.sort_values("Fichier").reset_index(drop=True))

# --- Quelques indicateurs globaux
nb_total = len(df_summary)
total_lignes = df_summary["Lignes"].sum()
total_colonnes = df_summary["Colonnes"].sum()

print(f"\n📈 Nombre total de bases : {nb_total}")
print(f"📊 Total de lignes (toutes bases confondues) : {total_lignes:,}")
print(f"🧱 Total de colonnes cumulées : {total_colonnes}")
print("🎯 Vérification visuelle effectuée : tu peux maintenant confirmer que tout est cohérent avant les fusions.")


In [ ]:
# === RÉCAPITULATIF GLOBAL DES BASES APRÈS NETTOYAGE ===
# Objectif :
#   - Afficher nombre de lignes / colonnes par base
#   - Montrer un aperçu (5 lignes × 8 colonnes)
#   - Vérifier cohérence et propreté globale

import pandas as pd
import csv
from pathlib import Path

# --- Dossier contenant les fichiers nettoyés
CLEAN_DIR = Path("data/clean")
files = sorted(CLEAN_DIR.glob("*.csv"))
print(f"📂 {len(files)} fichiers détectés dans {CLEAN_DIR}\n")

def detect_separator(path):
    """Détecte automatiquement le séparateur probable (, ou ;)"""
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        sample = f.read(4096)
    try:
        sep = csv.Sniffer().sniff(sample).delimiter
    except Exception:
        sep = ";"
    return sep

# --- Lecture et résumé global
summaries = []

for path in files:
    try:
        sep = detect_separator(path)
        df = pd.read_csv(path, sep=sep, encoding="utf-8", low_memory=False)

        # Informations de base
        lignes, colonnes = df.shape
        nan_rate = round(df.isna().mean().mean() * 100, 2)
        numeric_cols = len(df.select_dtypes(include=["number"]).columns)
        text_cols = len(df.select_dtypes(include=["object"]).columns)

        summaries.append({
            "Fichier": path.name,
            "Lignes": lignes,
            "Colonnes": colonnes,
            "Colonnes numériques": numeric_cols,
            "Colonnes texte": text_cols,
            "Taux NaN (%)": nan_rate
        })

        print(f"=== {path.name} ===")
        print(f"🔹 {lignes} lignes | {colonnes} colonnes")
        print(f"   📊 {numeric_cols} numériques | {text_cols} texte | NaN moyen : {nan_rate}%")
        print("   🔸 Aperçu des premières colonnes :")
        display(df.head(5).iloc[:, :min(8, colonnes)])  # 5 lignes × 8 colonnes max
        print("-" * 120)

    except Exception as e:
        print(f"⚠️ Impossible de lire {path.name} : {e}")
        print("-" * 100)

# --- Synthèse finale
print("\n📊 === SYNTHÈSE GLOBALE DES BASES ===")
df_summary = pd.DataFrame(summaries)
display(df_summary.sort_values("Fichier").reset_index(drop=True))

# --- Quelques indicateurs globaux
nb_total = len(df_summary)
total_lignes = df_summary["Lignes"].sum()
total_colonnes = df_summary["Colonnes"].sum()

print(f"\n📈 Nombre total de bases : {nb_total}")
print(f"📊 Total de lignes (toutes bases confondues) : {total_lignes:,}")
print(f"🧱 Total de colonnes cumulées : {total_colonnes}")
print("🎯 Vérification visuelle effectuée : tu peux maintenant confirmer que tout est cohérent avant les fusions.")
